In [ ]:
!pip3 install datasets
!pip3 install unsloth

In [ ]:
from datasets import load_dataset
dataset = load_dataset("sentence-transformers/eli5", split = "train")

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from unsloth import FastLanguageModel
import numpy as np
import pandas as pd
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/TEST')
os.environ['WANDB_MODE'] = 'disabled'
model_name = "./MODEL/llama3.2-1b"

In [ ]:
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
max_seq_length = 512
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
model = FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.11.11: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Unsloth 2024.11.11 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
def formatting_prompts_func(examples):
    questions = examples['question']
    answers = examples['answer']
    convos = []
    for question, answer in zip(questions, answers):
        convos.append([{"role": "user", "content": question}, {"role": "assistant", "content": answer}])
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

README.md:   0%|          | 0.00/2.24k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/113M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/325475 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset = dataset.train_test_split(test_size=0.2)

Map:   0%|          | 0/325475 [00:00<?, ? examples/s]

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

In [ ]:
def stream_answer(question, model=model):
    messages = [
        {"role": "user", "content": question},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")

    # Create attention mask
    attention_mask = torch.ones_like(inputs).to("cuda")
    answer = model.generate(
        input_ids = inputs,
        attention_mask = attention_mask,
        streamer = text_streamer,
        max_new_tokens = 100,
        use_cache = True
    )
    return answer

In [ ]:
import random
random.seed(42)
selected_indices = random.sample(range(len(dataset['test'])), 1200)
selected_dataset = dataset['test'].select(selected_indices)

questions = []
for i in range(1200):
  question = selected_dataset['question'][i]
  questions.append(question)
  #answer = stream_answer(question) uncomment to run, very long

Bulwer-L<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Can Conservation of Energy and the "many worlds" interpretation of Quantum Mechanics be reconsiled?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

This is a great question, and I have always thought about it.  The idea is that the wavefunction for a quantum system is a generalization of the concept of probability. If you think about it, a wavefunction is just a probability distribution. You can interpret this as a way of saying that the probability of something happening is proportional to its value. If you have a system where you can only measure a single value, the probability of that value is 1. But if you have a
Yes, they did. The [Mandeville Scrolls](_URL_0_) from the 10th century are a good example of this. They contain a large number of texts that describe fantastical worlds, including a 

In [ ]:
#copy last screen output from streamer
#texts = ''' '''

In [ ]:
#answers = texts.split("\n")
answers = pd.read_csv("qa_pairs2.csv")["answers"].to_list()

In [ ]:
!pip3 install rouge-score
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [ ]:
import re
def find_max_similarity(answer):
  segments = [s.strip() for s in re.split('[,.!?]', answer) if len(s.strip().split()) >= 2]
  max_score, s1, s2 = 0, "", ""
  if len(segments) > 0:
    for i in range(len(segments)):
      for j in range(i + 1, len(segments)):
        score = scorer.score(segments[i], segments[j])['rougeL'].fmeasure
        if score > max_score: max_score, s1, s2 = score, segments[i], segments[j]
  return (max_score, s1, s2)

In [ ]:
data = []
for i in range(1200):
  question, temp = questions[i], answers[i]
  answer = re.sub(r'\(_URL_\d+_\)', '', temp)
  last_period = answer.rfind('.')
  if last_period != -1: answer = answer[:last_period + 1].strip()
  else: answer = answer.strip()
  max_score, s1, s2 = find_max_similarity(answer)
  if max_score > 0.651: label = 0
  else: label = 1
  data.append({
      "questions": question,
      "answers": answer,
      "labels": label,
      "max_rouge_l": round(max_score, 3),
      "s1": s1,
      "s2": s2,
  })
df_QA = pd.DataFrame(data)

In [ ]:
df_QA.to_csv("qa_pairs2.csv", index = False)

In [ ]:
df_QA

,questions,answers,labels,max_rouge_l,s1,s2
0,"Can Conservation of Energy and the ""many world...","This is a great question, and I have always th...",1,0.333,This is a great question,a wavefunction is just a probability distribution
1,From 1700/ 1600 backwards - did people write s...,"Yes, they did. The [Mandeville Scrolls] from t...",1,0.250,They contain a large number of texts that desc...,The [Talmud] is another example of a written w...
2,Were there any great Sub-Saharan African Kingd...,The Kingdom of Kongo was a powerful and wealth...,0,1.000,The Kingdom of the Ashanti was known for its g...,The Kingdom of the Ashanti was known for its g...
3,What makes an allele dominant or recessive?,The answer to your question is that it depends...,1,0.545,A and B are recessive,and B is recessive to A
4,Why are there two Captain's Quarters on the US...,There is no evidence that the Constitution had...,0,1.000,The first captain's quarters was built in 1815,The first captain's quarters was built in 1815
...,...,...,...,...,...,...
1195,What is wrong with the Democratic Republic of ...,The DRC is a country with an incredibly poor e...,1,0.500,no manufacturing,no infrastructure
1196,What would happen to animals that sleep during...,"There are a few different ways to look at it, ...",1,0.476,but the simplest is that animals would have a ...,Most animals would not survive the winter
1197,Why were the Japanese so brutal in WW2?,I think you're misunderstanding the question. ...,1,0.588,They were the only country to have a navy that...,They were the only country to have a military ...
1198,Why do historians sometimes reference a second...,I'm not sure if this is the best way to explai...,0,0.667,Historians sometimes use secondary sources to ...,and I am using primary sources to look at prim...


In [ ]:
len(df_QA[df_QA["labels"]==1]),len(df_QA[df_QA["labels"]==0])

(567, 633)